In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [1]:
import soundfile as sf
import torch
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import librosa

In [2]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You sho

In [3]:
file_path = "/content/drive/MyDrive/data/songs/Ballin.flac" #customize it
audio_input, sample_rate = librosa.load(file_path, sr=16000)

In [ ]:
# librispeech_samples_ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")

# # load audio
# audio_input, sample_rate = sf.read(librispeech_samples_ds[0]["file"])
# inputs = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values

In [4]:
inputs = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values

In [19]:
inputs

tensor([[7.5996e-05, 7.5846e-05, 7.5893e-05,  ..., 8.9248e-05, 9.2605e-05,
         7.6219e-05]])

In [ ]:
logits = model(inputs).logits
predicted_ids = torch.argmax(logits, dim=-1)

# transcribe
transcription = processor.decode(predicted_ids[0])